# 221010 - iBioFoundry - Plate Prep - OT2
author: Camillo Moschner | version: 1.6 | date: 10.10.2022 | license = 

# 1- Metadata

In [1]:
metadata = {
    'protocolName': 'iBF_plate_prep',
    'author': 'Camillo Moschner <cm967@cam.ac.uk> / <camillo.moschner@gmail.com>',
    'description': 'Automated preparation of Plate Preps',
    'apiLevel': '2.12',
    'Date': '10.10.2022',
    'pipette_configuration':{'left':'p20_single_gen2',
                             'right':'p300_single_gen2'}
    }

# 2- Import Statements

In [2]:
import numpy as np
import pandas as pd
from IPython.display import display, clear_output, Audio, display, display_html
from datetime import date
import os
from itertools import product

from iBioFoundry_helper import *
from opentrons import protocol_api, execute, simulate

## Plate Pipetting Master Inputs
The following cell requires changing between experimental setups (and, for simplicity for the user, is supposed to be the only part of the protocol that requires any modification).

In [3]:
# establish whether you are performing a simulation (e.g. at the design stage) or are executing the script on the OT-2
script_mode = 'simulation' # 'simulation' or 'execution'
experiment_name = 'iGEM22_PBDAD_Hills_t5'
audiofeedback = False # True if you want to to receive audio-notification of a finished process, else change to Falsev

# resuspended colony info
resusp_colony_plate_format = 'opentrons_96_aluminumblock_generic_pcr_strip_200ul' # 'opentrons_96_aluminumblock_biorad_wellplate_200ul'
resusp_col_plate_column_usage = 'odd' # allows you to selectively only use 'odd'- or 'even'-numbered columns or 'all'; useful for 0.2 ml tube usage
# extra reagent info
reagents_rack_format  = 'opentrons_24_tuberack_eppendorf_1.5ml_safelock_snapcap'
# culture plate info
culture_plate_format = 'corning_384_wellplate_112ul_flat' #'corning_96_wellplate_360ul_flat'
culture_plate_column_usage = 'all'
# final reaction composition
cell_culture_vol = 50.0 # units: µl # 1:200 -> 0.25:50 -> 0.125:50 meaning 1:400
resusp_col_for_cell_culture_vol = 1.00 # units: µl -> prep by adding 7.5 ul O/N culture to 52.5 ul EZRDM

In [4]:
# establish sole Jupyter Notebook control 
if script_mode == 'execution':
    try:
        os.system("systemctl stop opentrons-robot-server")
        protocol = execute.get_protocol_api(metadata['apiLevel']);
    except:
        protocol = execute.get_protocol_api(metadata['apiLevel']);
    print("You are operating this script in exectution mode!\nYou should see the lights turn on and hear the gantry homing!")
    startNow()
elif script_mode == 'simulation':
    protocol = simulate.get_protocol_api(metadata['apiLevel']);
    print("You are operating this script in simulation mode!")
protocol.set_rail_lights(True)
protocol.home()

/Users/camillomoschner/.opentrons/robot_settings.json not found. Loading defaults
/Users/camillomoschner/.opentrons/deck_calibration.json not found. Loading defaults


You are operating this script in simulation mode!


# 3- Function Definitions

In [5]:
def smart_transfer_liquid(transfer_vol, aspiration_pos,dispensation_pos,
                          pick_up_tip=True, touch_tip_before=True,
                          cyclical_mix_before=None,mix_before_reps=2,mix_rate=0.5,
                          asp_bot_clearance=1.5,dispens_bot_clearance=1.5,
                          asp_rate=0.8,disp_rate=0.5,
                          cyclical_mix_after=None,mix_after=None,
                          dispens_top_offset=-2, blow_out=False, air_gap=False, touch_tip=True, touch_miniscus=None,
                          drop_tip=True):
    """Liquid transfer that automatically identifies which tip to use.
    """
    if (transfer_vol >= p20_left.min_volume) & (transfer_vol <= p20_left.max_volume):
        pipette_to_use = p20_left
    elif (transfer_vol > p300_right.min_volume):
        pipette_to_use = p300_right
    else:
        print(f"WARNING: transfer volume not manageable")
    if pick_up_tip==True:
        pipette_to_use.pick_up_tip()
    if cyclical_mix_before != None: # give tuple (repetitions, mix_volume, well_bottom_clearance)
        for i in np.arange(cyclical_mix_before[0]):
            pipette_to_use.aspirate(cyclical_mix_before[1],aspiration_pos.bottom(cyclical_mix_before[2]))
            pipette_to_use.dispense(cyclical_mix_before[1],aspiration_pos.bottom(cyclical_mix_before[3]))
    if mix_before_reps != 0:
        pipette_to_use.mix(mix_before_reps,transfer_vol*0.8,aspiration_pos.bottom(asp_bot_clearance),mix_rate)
    # aspiration
    pipette_to_use.aspirate(transfer_vol,aspiration_pos.bottom(asp_bot_clearance), asp_rate)
    pipette_to_use.move_to(aspiration_pos.top())
    if air_gap != False:
        pipette_to_use.air_gap(air_gap)
    protocol.delay(seconds=1)
    if touch_tip_before == True:
        pipette_to_use.touch_tip(v_offset=-2, speed=150)
    # dispensation
    pipette_to_use.dispense(transfer_vol,dispensation_pos.bottom(dispens_bot_clearance),disp_rate)
    if cyclical_mix_after != None: # give tuple (repetitions, mix_volume, asp- , disp-well_bottom_clearance)
        for i in np.arange(cyclical_mix_after[0]):
            pipette_to_use.aspirate(cyclical_mix_after[1],dispensation_pos.bottom(cyclical_mix_after[2]))
            pipette_to_use.dispense(cyclical_mix_after[1],dispensation_pos.bottom(cyclical_mix_after[3]))
    if mix_after != None:
        pipette_to_use.mix(mix_after[0],mix_after[1], dispensation_pos.bottom(mix_after[2]),mix_rate)
    pipette_to_use.move_to(dispensation_pos.top())
    protocol.delay(seconds=1)
    if blow_out == True:
        pipette_to_use.blow_out(dispensation_pos.top(dispens_top_offset))
    if touch_tip == True:
        pipette_to_use.touch_tip(v_offset=dispens_top_offset, speed=150)
    if touch_miniscus != None:
        protocol.max_speeds['Z'] = 10  # limit z axis to 50 mm/s to avoid solution adhereing to outer tip surface
        pipette_to_use.move_to(dispensation_pos.bottom(touch_miniscus))
        protocol.max_speeds['Z'] = None # reset z axis speed
    if drop_tip==True:
        pipette_to_use.drop_tip()
        
def smart_distribute_liquid(single_distr_vol, aspiration_pos,dispensation_pos_list,
                            asp_bot_clearance=1.1,dispens_bot_clearance=1.8,
                            asp_flow_rate = 1,disp_flow_rate = 1, new_disposal_volume=None,
                            touch_tip_bool=True,blow_out_bool=False):
    """Liquid distribution from one aspiration well into multiple destination wells that automatically identifies which pipette to use.
    """
    if (single_distr_vol >= p20_left.min_volume) & (single_distr_vol <= p20_left.max_volume):
        pipette_to_use = p20_left
    elif (single_distr_vol > p300_right.min_volume):
        pipette_to_use = p300_right
    # save original/default flow rates
    original_asp_flow_rate = pipette_to_use.flow_rate.aspirate
    original_disp_flow_rate = pipette_to_use.flow_rate.dispense
    # change flow rates if desired
    pipette_to_use.flow_rate.aspirate = original_asp_flow_rate *asp_flow_rate
    pipette_to_use.flow_rate.dispense = original_disp_flow_rate *disp_flow_rate
    # change well_bottom_clearances if desired
    pipette_to_use.well_bottom_clearance.aspirate = asp_bot_clearance
    pipette_to_use.well_bottom_clearance.dispense = dispens_bot_clearance
    # execute
    if new_disposal_volume != None:
        pipette_to_use.distribute(single_distr_vol, aspiration_pos,dispensation_pos_list, 
                                  touch_tip=touch_tip_bool,disposal_volume=new_disposal_volume,blow_out=blow_out_bool)
    else:
        pipette_to_use.distribute(single_distr_vol, aspiration_pos,dispensation_pos_list,
                                  touch_tip=touch_tip_bool,blow_out=blow_out_bool)
    # set well_bottom_clearances back to default
    pipette_to_use.well_bottom_clearance.dispense = 1
    pipette_to_use.well_bottom_clearance.aspirate = 1
    # set flow rates back to default
    pipette_to_use.flow_rate.aspirate = original_asp_flow_rate
    pipette_to_use.flow_rate.dispense = original_disp_flow_rate

def flatten(l):
    return [item for sublist in l for item in sublist]

---
# 4- Deck assignments

In [6]:
"""Deck creation"""
deck_slot_list =list(np.arange(1,12))
deck_slot_list.append('bin')
deck_slot_df = pd.DataFrame( np.flip(np.array(deck_slot_list).reshape(4,3), axis=0) )

## i. Labware
Resource: [Opentrons Labware Library Webpage](https://labware.opentrons.com/?_gl=1*1iyc1t4*_ga*MTU1MTM1NzU5MS4xNjE2OTczMzY5*_ga_GNSMNLW4RY*MTYzMDYxNzY1NS4yMC4wLjE2MzA2MTc2NTUuMA..&_ga=2.110416899.767557364.1630617656-1551357591.1616973369)
### a. Modules

In [7]:
temp_mod_name  = 'temperature module gen2'
temp_mod = protocol.load_module(temp_mod_name,'3')
deck_slot_df.iloc[3,2] = temp_mod_name

### b. Racks & Plates

In [8]:
tiprack20a = protocol.load_labware('opentrons_96_tiprack_20ul','8')
tiprack20b = protocol.load_labware('opentrons_96_tiprack_20ul','11')
tiprack300a = protocol.load_labware('opentrons_96_tiprack_300ul','10')
# tuberacks & plates
resusp_colony_plate = protocol.load_labware(resusp_colony_plate_format,'2') # opentrons_96_aluminumblock_biorad_wellplate_200ul
culture_plate = temp_mod.load_labware(culture_plate_format,'3')
reagents_rack = protocol.load_labware(reagents_rack_format,'1') #  biorad_96_wellplate_200ul_pcr
large_vol_rack = protocol.load_labware('opentrons_15_tuberack_falcon_15ml_conical','4')   # opentrons_6_tuberack_falcon_50ml_conical

In [9]:
labware_list = [tiprack20a,tiprack20b, tiprack300a,
                resusp_colony_plate,culture_plate, reagents_rack,large_vol_rack]
_ =[update_labware(deck_slot_df, labware_item) for labware_item in labware_list]
# visually inspect deck
print("\nFinal OT-2 Deck Configuration:");(deck_slot_df)


Final OT-2 Deck Configuration:


,0,1,2
0,10-opentrons_96_tiprack_300ul,11-opentrons_96_tiprack_20ul,bin
1,7,8-opentrons_96_tiprack_20ul,9
2,4-opentrons_15_tuberack_falcon_15ml_conical,5,6
3,1-opentrons_24_tuberack_eppendorf_1.5ml_safelo...,2-opentrons_96_aluminumblock_generic_pcr_strip...,temperature module gen2


## ii. Pipettes

Resource: [Opentrons Pipettes API Webpage](https://docs.opentrons.com/v2/new_pipette.html)

In [10]:
p20_left = protocol.load_instrument('p20_single_gen2','left', tip_racks=[tiprack20a,tiprack20b])
p300_right = protocol.load_instrument('p300_single_gen2','right', tip_racks=[tiprack300a])

### In-Notebook Calibration

In [11]:
#if in_jupyter_calibration == True:
#import opentrons
pipette_to_calibrate = p20_left # p20_left | p300_right
labware_to_calibrate_to = resusp_colony_plate # resusp_colony_plate | culture_plate | reagents_rack | large_vol_rack
labware_position_to_test = 'A1'

In [12]:
# step 0: pick up tip for the pipette you want to calibrate and move to the labware to be calibrated
pipette_to_calibrate.pick_up_tip()
pipette_to_calibrate.move_to(labware_to_calibrate_to[labware_position_to_test].top())

<InstrumentContext: p20_single_v2.0 in LEFT>

In [13]:
# step 1: set_offset based on identifying in which direction the tip needs to move
# (can be an iderative process as this cell immediately moves to the new position)
labware_to_calibrate_to.set_offset(x=(0.),y=(0.),z=(0.))
pipette_to_calibrate.move_to(labware_to_calibrate_to[labware_position_to_test].top())

<InstrumentContext: p20_single_v2.0 in LEFT>

In [14]:
pipette_to_calibrate.move_to(labware_to_calibrate_to['H1'].top())

<InstrumentContext: p20_single_v2.0 in LEFT>

In [15]:
# step 2: return tip (if you haven'crashed it
pipette_to_calibrate.return_tip()

<InstrumentContext: p20_single_v2.0 in LEFT>

---
# 5- LH Step Calculations 
## Design Allocation

Define your experimental conditions:

In [16]:
# constructs
samples_cell_levels = ['y']
construct_levels = ['PB1','PB2','PB3',
                    'PB4','PB5','PB6','PB7']
# hosts
host = ['Marionette','TOP10']
# environments
media = ['EZRDM']
abx_c_levels = ['40ug'] # Kanamycin; units: ug/mL
inducer_c_levels = ['0uM','10uM','25uM',
                    '50uM', '75uM','100uM',
                    '125uM','150um'] # arabinose; units: micromolar
techn_replicates_no = 3

Test out how many combinations of runs will be generated using different controls, replicate numbers, and testing conditions:

In [17]:
NC = [[('n', 'nan', 'NTC', 'EZRDM', inducer_c_levels[0],'40ug')]*techn_replicates_no]  + [
     [('n', 'nan','NTC', 'EZRDM', '1000uM','40ug')]*techn_replicates_no] + [
    [('n', 'nan','NTC', 'EZRDM', inducer_c_levels[-1],'40ug')]*techn_replicates_no] + [
    [('y', 'Marionette', 'just_cells', 'EZRDM', '0ug'+inducer_c_levels[0],'0ug')]*techn_replicates_no]  + [
    [('y', 'Marionette', 'just_cells', 'EZRDM', '0ug'+'1000uM','0ug')]*techn_replicates_no] + [
    [('y', 'Marionette', 'just_cells', 'EZRDM', '0ug'+inducer_c_levels[-1],'0ug')]*techn_replicates_no] 

#NC  = [[x]*techn_replicates_no for x in list(product(['n'], ['nan'], ['NC'], media, inducer_c_levels, abx_c_levels))]
samples = [[x]*techn_replicates_no for x in list(product(samples_cell_levels, host, construct_levels, media, inducer_c_levels, abx_c_levels))]
complete_run_list = flatten(NC + samples)
complete_run_df = pd.DataFrame( complete_run_list, columns=['cells', 'host', 'samples', 'media', 'ind_c','abx_c'])
complete_run_df['replicate']  = (len(NC)+len(samples))*list(range(techn_replicates_no))
complete_run_df['ID'] = complete_run_df.host+complete_run_df.samples+complete_run_df.ind_c.apply(lambda x: f"|{x}")+complete_run_df.replicate.apply(lambda x: f"|{x}")
complete_run_df#.head(50)

,cells,host,samples,media,ind_c,abx_c,replicate,ID
0,n,nan,NTC,EZRDM,0uM,40ug,0,nanNTC|0uM|0
1,n,nan,NTC,EZRDM,0uM,40ug,1,nanNTC|0uM|1
2,n,nan,NTC,EZRDM,0uM,40ug,2,nanNTC|0uM|2
3,n,nan,NTC,EZRDM,1000uM,40ug,0,nanNTC|1000uM|0
4,n,nan,NTC,EZRDM,1000uM,40ug,1,nanNTC|1000uM|1
...,...,...,...,...,...,...,...,...
349,y,TOP10,PB7,EZRDM,125uM,40ug,1,TOP10PB7|125uM|1
350,y,TOP10,PB7,EZRDM,125uM,40ug,2,TOP10PB7|125uM|2
351,y,TOP10,PB7,EZRDM,150um,40ug,0,TOP10PB7|150um|0
352,y,TOP10,PB7,EZRDM,150um,40ug,1,TOP10PB7|150um|1


## Object Creation

In [18]:
# create overnight cell culture 8-strip object
on_cultures_rack_obj = Plate(96, 'O/N cultures aliquoted in 8-strip tubes', only_columns=resusp_col_plate_column_usage)
on_cultures_rack_obj.layout = extend_fill_plate_df_with_list(on_cultures_rack_obj.layout, list(complete_run_df.samples.unique()),
                                                             fill_start_position='A1', fill_first='columns')
# create media rack object
media_rack_obj = Plate(15, 'media in 15ml Falkon tubes')
media_rack_obj.layout = extend_fill_plate_df_with_list(media_rack_obj.layout, list(complete_run_df.ind_c.unique()),
                                                       fill_start_position='A1', fill_first='columns')
# create cell culture media plate object
culture_plate_media_obj = Plate(int(identify_plate(culture_plate_format, 'plate_format')), 'cell culture plate (abx concentrations)')
culture_plate_media_obj.layout = extend_fill_plate_df_with_list(culture_plate_media_obj.layout, complete_run_df.ind_c.to_list(),
                                                                fill_start_position='A1', fill_first='columns')
# create combined cell culture plate object
culture_plate_obj = Plate(int(identify_plate(culture_plate_format, 'plate_format')), 'cell culture plate (ID)')
culture_plate_obj.layout = extend_fill_plate_df_with_list(culture_plate_obj.layout, complete_run_df.ID.to_list(),
                                                          fill_start_position='A1', fill_first='columns')

In [19]:
complete_run_df['well'] = culture_plate_obj.occupied_wells.keys()
complete_run_df['media_source'] = complete_run_df.ind_c.apply(lambda x: large_vol_rack[find_df_coordinates(media_rack_obj.layout, x)])
complete_run_df['sample_source'] = complete_run_df.samples.apply(lambda x: resusp_colony_plate[find_df_coordinates(on_cultures_rack_obj.layout, x)])
complete_run_df['destination'] = complete_run_df.well.apply(lambda x: culture_plate[x])
complete_run_df['prep_date'] = [str(date.today()) for x in range(len(complete_run_df))]
complete_run_df.set_index('well',inplace=True)
complete_run_df

,cells,host,samples,media,ind_c,abx_c,replicate,ID,media_source,sample_source,destination,prep_date
well,,,,,,,,,,,,
A1,n,nan,NTC,EZRDM,0uM,40ug,0,nanNTC|0uM|0,A1 of Opentrons 15 Tube Rack with Falcon 15 mL...,A1 of Opentrons 96 Well Aluminum Block with Ge...,A1 of 3 on Temperature Module GEN2 on 3,2023-06-04
B1,n,nan,NTC,EZRDM,0uM,40ug,1,nanNTC|0uM|1,A1 of Opentrons 15 Tube Rack with Falcon 15 mL...,A1 of Opentrons 96 Well Aluminum Block with Ge...,B1 of 3 on Temperature Module GEN2 on 3,2023-06-04
C1,n,nan,NTC,EZRDM,0uM,40ug,2,nanNTC|0uM|2,A1 of Opentrons 15 Tube Rack with Falcon 15 mL...,A1 of Opentrons 96 Well Aluminum Block with Ge...,C1 of 3 on Temperature Module GEN2 on 3,2023-06-04
D1,n,nan,NTC,EZRDM,1000uM,40ug,0,nanNTC|1000uM|0,B1 of Opentrons 15 Tube Rack with Falcon 15 mL...,A1 of Opentrons 96 Well Aluminum Block with Ge...,D1 of 3 on Temperature Module GEN2 on 3,2023-06-04
E1,n,nan,NTC,EZRDM,1000uM,40ug,1,nanNTC|1000uM|1,B1 of Opentrons 15 Tube Rack with Falcon 15 mL...,A1 of Opentrons 96 Well Aluminum Block with Ge...,E1 of 3 on Temperature Module GEN2 on 3,2023-06-04
...,...,...,...,...,...,...,...,...,...,...,...,...
N22,y,TOP10,PB7,EZRDM,125uM,40ug,1,TOP10PB7|125uM|1,C4 of Opentrons 15 Tube Rack with Falcon 15 mL...,A3 of Opentrons 96 Well Aluminum Block with Ge...,N22 of 3 on Temperature Module GEN2 on 3,2023-06-04
O22,y,TOP10,PB7,EZRDM,125uM,40ug,2,TOP10PB7|125uM|2,C4 of Opentrons 15 Tube Rack with Falcon 15 mL...,A3 of Opentrons 96 Well Aluminum Block with Ge...,O22 of 3 on Temperature Module GEN2 on 3,2023-06-04
P22,y,TOP10,PB7,EZRDM,150um,40ug,0,TOP10PB7|150um|0,C1 of Opentrons 15 Tube Rack with Falcon 15 mL...,A3 of Opentrons 96 Well Aluminum Block with Ge...,P22 of 3 on Temperature Module GEN2 on 3,2023-06-04


## - Visual Deck Inspection

In [20]:
complete_run_df.to_csv("221007_decryption_spreadsheet.csv")

In [21]:
on_cultures_rack_obj_styler = on_cultures_rack_obj.layout.style.set_table_attributes("style='display:inline'").set_caption(f"Layout of {on_cultures_rack_obj.name} ({resusp_colony_plate.parent})")
media_rack_styler = media_rack_obj.layout.style.set_table_attributes("style='display:inline'").set_caption(f"Layout of {media_rack_obj.name} ({large_vol_rack.parent})")
culture_plate_media_styler = culture_plate_media_obj.layout.style.set_table_attributes("style='display:inline'").set_caption(f"Layout of {culture_plate_media_obj.name} ({culture_plate.parent})")
culture_plate_styler = culture_plate_obj.layout.style.set_table_attributes("style='display:inline'").set_caption(f"Layout of {culture_plate_obj.name} ({culture_plate.parent})")
space = "\xa0"*5
display_html(media_rack_styler._repr_html_()+space+on_cultures_rack_obj_styler._repr_html_()+space+culture_plate_media_styler._repr_html_()+space+culture_plate_styler._repr_html_(), raw=True)
print(f"\n{'-'*100}\n")

Layout of media in 15ml Falkon tubes (4) 
 
 
   
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 A 
 0uM 
 0ug0uM 
 10uM 
 75uM 
 0 
 
 
 B 
 1000uM 
 0ug1000uM 
 25uM 
 100uM 
 0 
 
 
 C 
 150um 
 0ug150um 
 50uM 
 125uM 
 0 
 
 
 
     
 
 Layout of O/N cultures aliquoted in 8-strip tubes (2) 
 
 
   
 1 
 3 
 5 
 7 
 9 
 11 
 
 
 
 
 A 
 NTC 
 PB7 
 0 
 0 
 0 
 0 
 
 
 B 
 just_cells 
 0 
 0 
 0 
 0 
 0 
 
 
 C 
 PB1 
 0 
 0 
 0 
 0 
 0 
 
 
 D 
 PB2 
 0 
 0 
 0 
 0 
 0 
 
 
 E 
 PB3 
 0 
 0 
 0 
 0 
 0 
 
 
 F 
 PB4 
 0 
 0 
 0 
 0 
 0 
 
 
 G 
 PB5 
 0 
 0 
 0 
 0 
 0 
 
 
 H 
 PB6 
 0 
 0 
 0 
 0 
 0 
 
 
 
     
 
 Layout of cell culture plate (abx concentrations) (Temperature Module GEN2 on 3) 
 
 
   
 1 
 2 
 3 
 4 
 5 
 6 
 7 
 8 
 9 
 10 
 11 
 12 
 13 
 14 
 15 
 16 
 17 
 18 
 19 
 20 
 21 
 22 
 23 
 24 
 
 
 
 
 A 
 0uM 
 0ug150um 
 75uM 
 25uM 
 150um 
 75uM 
 25uM 
 150um 
 75uM 
 25uM 
 150um 
 75uM 
 25uM 
 150um 
 75uM 
 25uM 
 150um 
 75uM 
 25uM 
 150um 
 75uM 
 25uM 
 150um 
 0 
 
 
 B 
 0uM 
 0ug150um 
 100uM 
 25uM 
 150um 
 100uM 
 25uM 
 150um 
 100uM 
 25uM 
 150um 
 100uM 
 25uM 
 150um 
 100uM 
 25uM 
 150um 
 100uM 
 25uM 
 150um 
 100uM 
 25uM 
 150um 
 0 
 
 
 C 
 0uM 
 0uM 
 100uM 
 25uM 
 0uM 
 100uM 
 25uM 
 0uM 
 100uM 
 25uM 
 0uM 
 100uM 
 25uM 
 0uM 
 100uM 
 25uM 
 0uM 
 100uM 
 25uM 
 0uM 
 100uM 
 25uM 
 0 
 0 
 
 
 D 
 1000uM 
 0uM 
 100uM 
 50uM 
 0uM 
 100uM 
 50uM 
 0uM 
 100uM 
 50uM 
 0uM 
 100uM 
 50uM 
 0uM 
 100uM 
 50uM 
 0uM 
 100uM 
 50uM 
 0uM 
 100uM 
 50uM 
 0 
 0 
 
 
 E 
 1000uM 
 0uM 
 125uM 
 50uM 
 0uM 
 125uM 
 50uM 
 0uM 
 125uM 
 50uM 
 0uM 
 125uM 
 50uM 
 0uM 
 125uM 
 50uM 
 0uM 
 125uM 
 50uM 
 0uM 
 125uM 
 50uM 
 0 
 0 
 
 
 F 
 1000uM 
 10uM 
 125uM 
 50uM 
 10uM 
 125uM 
 50uM 
 10uM 
 125uM 
 50uM 
 10uM 
 125uM 
 50uM 
 10uM 
 125uM 
 50uM 
 10uM 
 125uM 
 50uM 
 10uM 
 125uM 
 50uM 
 0 
 0 
 
 
 G 
 150um 
 10uM 
 125uM 
 75uM 
 10uM 
 125uM 
 75uM 
 10uM 
 125uM 
 75uM 
 10uM 
 125uM 
 75uM 
 10uM 
 125uM 
 75uM 
 10uM 
 125uM 
 75uM 
 10uM 
 125uM 
 75uM 
 0 
 0 
 
 
 H 
 150um 
 10uM 
 150um 
 75uM 
 10uM 
 150um 
 75uM 
 10uM 
 150um 
 75uM 
 10uM 
 150um 
 75uM 
 10uM 
 150um 
 75uM 
 10uM 
 150um 
 75uM 
 10uM 
 150um 
 75uM 
 0 
 0 
 
 
 I 
 150um 
 25uM 
 150um 
 75uM 
 25uM 
 150um 
 75uM 
 25uM 
 150um 
 75uM 
 25uM 
 150um 
 75uM 
 25uM 
 150um 
 75uM 
 25uM 
 150um 
 75uM 
 25uM 
 150um 
 75uM 
 0 
 0 
 
 
 J 
 0ug0uM 
 25uM 
 150um 
 100uM 
 25uM 
 150um 
 100uM 
 25uM 
 150um 
 100uM 
 25uM 
 150um 
 100uM 
 25uM 
 150um 
 100uM 
 25uM 
 150um 
 100uM 
 25uM 
 150um 
 100uM 
 0 
 0 
 
 
 K 
 0ug0uM 
 25uM 
 0uM 
 100uM 
 25uM 
 0uM 
 100uM 
 25uM 
 0uM 
 100uM 
 25uM 
 0uM 
 100uM 
 25uM 
 0uM 
 100uM 
 25uM 
 0uM 
 100uM 
 25uM 
 0uM 
 100uM 
 0 
 0 
 
 
 L 
 0ug0uM 
 50uM 
 0uM 
 100uM 
 50uM 
 0uM 
 100uM 
 50uM 
 0uM 
 100uM 
 50uM 
 0uM 
 100uM 
 50uM 
 0uM 
 100uM 
 50uM 
 0uM 
 100uM 
 50uM 
 0uM 
 100uM 
 0 
 0 
 
 
 M 
 0ug1000uM 
 50uM 
 0uM 
 125uM 
 50uM 
 0uM 
 125uM 
 50uM 
 0uM 
 125uM 
 50uM 
 0uM 
 125uM 
 50uM 
 0uM 
 125uM 
 50uM 
 0uM 
 125uM 
 50uM 
 0uM 
 125uM 
 0 
 0 
 
 
 N 
 0ug1000uM 
 50uM 
 10uM 
 125uM 
 50uM 
 10uM 
 125uM 
 50uM 
 10uM 
 125uM 
 50uM 
 10uM 
 125uM 
 50uM 
 10uM 
 125uM 
 50uM 
 10uM 
 125uM 
 50uM 
 10uM 
 125uM 
 0 
 0 
 
 
 O 
 0ug1000uM 
 75uM 
 10uM 
 125uM 
 75uM 
 10uM 
 125uM 
 75uM 
 10uM 
 125uM 
 75uM 
 10uM 
 125uM 
 75uM 
 10uM 
 125uM 
 75uM 
 10uM 
 125uM 
 75uM 
 10uM 
 125uM 
 0 
 0 
 
 
 P 
 0ug150um 
 75uM 
 10uM 
 150um 
 75uM 
 10uM 
 150um 
 75uM 
 10uM 
 150um 
 75uM 
 10uM 
 150um 
 75uM 
 10uM 
 150um 
 75uM 
 10uM 
 150um 
 75uM 
 10uM 
 150um 
 0 
 0 
 
 
 
     
 
 Layout of cell culture plate (ID) (Temperature Module GEN2 on 3) 
 
 
   
 1 
 2 
 3 
 4 
 5 
 6 
 7 
 8 
 9 
 10 
 11 
 12 
 13 
 14 
 15 
 16 
 17 
 18 
 19 
 20 
 21 
 22 
 23 
 24 
 
 
 
 
 A 
 nanNTC|0uM|0 
 Marionettejust_cells|0ug150um|1 
 MarionettePB1|75uM|2 
 MarionettePB2|25uM|0 
 MarionettePB2|150um|1 
 Ma


----------------------------------------------------------------------------------------------------



## Media Preparation

In [23]:
print(f"This experiment requires...")
for idx, unique_abx_c in enumerate(complete_run_df.ind_c.unique()):
    sample_no=len(complete_run_df.loc[complete_run_df.ind_c==unique_abx_c])
    print(f" - {idx}) {sample_no} samples of {unique_abx_c}")
    if sample_no*cell_culture_vol < 5_000:
        print(f"     {sample_no*cell_culture_vol}")

This experiment requires...
 - 0) 45 samples of 0uM
     2250.0
 - 1) 3 samples of 1000uM
     150.0
 - 2) 45 samples of 150um
     2250.0
 - 3) 3 samples of 0ug0uM
     150.0
 - 4) 3 samples of 0ug1000uM
     150.0
 - 5) 3 samples of 0ug150um
     150.0
 - 6) 42 samples of 10uM
     2100.0
 - 7) 42 samples of 25uM
     2100.0
 - 8) 42 samples of 50uM
     2100.0
 - 9) 42 samples of 75uM
     2100.0
 - 10) 42 samples of 100uM
     2100.0
 - 11) 42 samples of 125uM
     2100.0


In [24]:
c1 = 1_000_000 # units: uM
c1_alt = 100_000 # units: uM
v2 = 3_000 # units: ul
c2s_list = sorted([int(x[:-2]) for x in complete_run_df.ind_c.unique() if "g" not in x])

for c2 in c2s_list[1:]:
    vol_opt1 = c2*v2/c1
    print(f"For medium stock {c2} uM: -> {vol_opt1} ul into {v2} - with {int(c1/1_000_000)} M stock")
    if vol_opt1 < 1:
        print(f"        or alternative: -> {c2*v2/c1_alt} ul into {v2} - with {int(c1_alt/1000)} mM stock\n")

For medium stock 10 uM: -> 0.03 ul into 3000 - with 1 M stock
        or alternative: -> 0.3 ul into 3000 - with 100 mM stock

For medium stock 25 uM: -> 0.075 ul into 3000 - with 1 M stock
        or alternative: -> 0.75 ul into 3000 - with 100 mM stock

For medium stock 50 uM: -> 0.15 ul into 3000 - with 1 M stock
        or alternative: -> 1.5 ul into 3000 - with 100 mM stock

For medium stock 75 uM: -> 0.225 ul into 3000 - with 1 M stock
        or alternative: -> 2.25 ul into 3000 - with 100 mM stock

For medium stock 100 uM: -> 0.3 ul into 3000 - with 1 M stock
        or alternative: -> 3.0 ul into 3000 - with 100 mM stock

For medium stock 125 uM: -> 0.375 ul into 3000 - with 1 M stock
        or alternative: -> 3.75 ul into 3000 - with 100 mM stock

For medium stock 150 uM: -> 0.45 ul into 3000 - with 1 M stock
        or alternative: -> 4.5 ul into 3000 - with 100 mM stock

For medium stock 1000 uM: -> 3.0 ul into 3000 - with 1 M stock


## Cell Preparation

In [25]:
print(f"This experiment requires...")
for unique_samples in complete_run_df.samples.unique():
    sample_no=len(complete_run_df.loc[complete_run_df.samples==unique_samples])
    print(f" - {sample_no} samples of {unique_samples}")
    print(sample_no*resusp_col_for_cell_culture_vol)

This experiment requires...
 - 9 samples of NTC
9.0
 - 9 samples of just_cells
9.0
 - 48 samples of PB1
48.0
 - 48 samples of PB2
48.0
 - 48 samples of PB3
48.0
 - 48 samples of PB4
48.0
 - 48 samples of PB5
48.0
 - 48 samples of PB6
48.0
 - 48 samples of PB7
48.0


## Summary Spreadsheets

In [26]:
complete_run_df#.to_csv(f"_decryption_spreadsheet.csv")

,cells,host,samples,media,ind_c,abx_c,replicate,ID,media_source,sample_source,destination,prep_date
well,,,,,,,,,,,,
A1,n,nan,NTC,EZRDM,0uM,40ug,0,nanNTC|0uM|0,A1 of Opentrons 15 Tube Rack with Falcon 15 mL...,A1 of Opentrons 96 Well Aluminum Block with Ge...,A1 of 3 on Temperature Module GEN2 on 3,2023-06-04
B1,n,nan,NTC,EZRDM,0uM,40ug,1,nanNTC|0uM|1,A1 of Opentrons 15 Tube Rack with Falcon 15 mL...,A1 of Opentrons 96 Well Aluminum Block with Ge...,B1 of 3 on Temperature Module GEN2 on 3,2023-06-04
C1,n,nan,NTC,EZRDM,0uM,40ug,2,nanNTC|0uM|2,A1 of Opentrons 15 Tube Rack with Falcon 15 mL...,A1 of Opentrons 96 Well Aluminum Block with Ge...,C1 of 3 on Temperature Module GEN2 on 3,2023-06-04
D1,n,nan,NTC,EZRDM,1000uM,40ug,0,nanNTC|1000uM|0,B1 of Opentrons 15 Tube Rack with Falcon 15 mL...,A1 of Opentrons 96 Well Aluminum Block with Ge...,D1 of 3 on Temperature Module GEN2 on 3,2023-06-04
E1,n,nan,NTC,EZRDM,1000uM,40ug,1,nanNTC|1000uM|1,B1 of Opentrons 15 Tube Rack with Falcon 15 mL...,A1 of Opentrons 96 Well Aluminum Block with Ge...,E1 of 3 on Temperature Module GEN2 on 3,2023-06-04
...,...,...,...,...,...,...,...,...,...,...,...,...
N22,y,TOP10,PB7,EZRDM,125uM,40ug,1,TOP10PB7|125uM|1,C4 of Opentrons 15 Tube Rack with Falcon 15 mL...,A3 of Opentrons 96 Well Aluminum Block with Ge...,N22 of 3 on Temperature Module GEN2 on 3,2023-06-04
O22,y,TOP10,PB7,EZRDM,125uM,40ug,2,TOP10PB7|125uM|2,C4 of Opentrons 15 Tube Rack with Falcon 15 mL...,A3 of Opentrons 96 Well Aluminum Block with Ge...,O22 of 3 on Temperature Module GEN2 on 3,2023-06-04
P22,y,TOP10,PB7,EZRDM,150um,40ug,0,TOP10PB7|150um|0,C1 of Opentrons 15 Tube Rack with Falcon 15 mL...,A3 of Opentrons 96 Well Aluminum Block with Ge...,P22 of 3 on Temperature Module GEN2 on 3,2023-06-04


In [27]:
temp_mod.set_temperature(6)
if audiofeedback == True:
    allDone()

---
# 6- Execution


## i. Media Distribution

In [28]:
%%time
print(f"Distributing media...\n")
for unique_medium in complete_run_df.media_source.unique()[8:]:
    current_medium_info_df = complete_run_df.loc[complete_run_df.media_source==unique_medium]
    current_medium_destinations_list = current_medium_info_df.destination.to_list()
    print(f" - {cell_culture_vol} ul {current_medium_info_df.ind_c.iloc[0]} arabinose ({unique_medium.well_name})\n          -> culture plate {[pos.well_name for pos in current_medium_destinations_list]}")
    smart_distribute_liquid(cell_culture_vol, unique_medium, current_medium_destinations_list,
                            asp_bot_clearance=2,dispens_bot_clearance=2.,
                            asp_flow_rate = 1,disp_flow_rate = 1, new_disposal_volume=0,
                            touch_tip_bool=False,blow_out_bool=False)

Distributing media...

 - 50.0 ul 50uM arabinose (C3)
          -> culture plate ['L2', 'M2', 'N2', 'D4', 'E4', 'F4', 'L5', 'M5', 'N5', 'D7', 'E7', 'F7', 'L8', 'M8', 'N8', 'D10', 'E10', 'F10', 'L11', 'M11', 'N11', 'D13', 'E13', 'F13', 'L14', 'M14', 'N14', 'D16', 'E16', 'F16', 'L17', 'M17', 'N17', 'D19', 'E19', 'F19', 'L20', 'M20', 'N20', 'D22', 'E22', 'F22']
 - 50.0 ul 75uM arabinose (A4)
          -> culture plate ['O2', 'P2', 'A3', 'G4', 'H4', 'I4', 'O5', 'P5', 'A6', 'G7', 'H7', 'I7', 'O8', 'P8', 'A9', 'G10', 'H10', 'I10', 'O11', 'P11', 'A12', 'G13', 'H13', 'I13', 'O14', 'P14', 'A15', 'G16', 'H16', 'I16', 'O17', 'P17', 'A18', 'G19', 'H19', 'I19', 'O20', 'P20', 'A21', 'G22', 'H22', 'I22']
 - 50.0 ul 100uM arabinose (B4)
          -> culture plate ['B3', 'C3', 'D3', 'J4', 'K4', 'L4', 'B6', 'C6', 'D6', 'J7', 'K7', 'L7', 'B9', 'C9', 'D9', 'J10', 'K10', 'L10', 'B12', 'C12', 'D12', 'J13', 'K13', 'L13', 'B15', 'C15', 'D15', 'J16', 'K16', 'L16', 'B18', 'C18', 'D18', 'J19', 'K19', 'L19', 'B21

## ii. Cell Distribution

In [29]:
%%time
print(f"Distributing samples...\n")
for unique_sample in complete_run_df.sample_source.unique():
    current_sample_info_df = complete_run_df.loc[complete_run_df.sample_source==unique_sample]
    current_sample_destinations_list = current_sample_info_df.destination.to_list()
    print(f" - {resusp_col_for_cell_culture_vol} ul {current_sample_info_df.samples.iloc[0]} ({unique_sample.well_name})\n          -> culture plate {[pos.well_name for pos in current_sample_destinations_list]}")
    smart_distribute_liquid(resusp_col_for_cell_culture_vol, unique_sample, current_sample_destinations_list,
                            asp_bot_clearance=2,dispens_bot_clearance=2.,
                            asp_flow_rate = 0.9,disp_flow_rate = 0.9, new_disposal_volume=1,
                            touch_tip_bool=False,blow_out_bool=False)

Distributing samples...

 - 1.0 ul NTC (A1)
          -> culture plate ['A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1', 'I1']
 - 1.0 ul just_cells (B1)
          -> culture plate ['J1', 'K1', 'L1', 'M1', 'N1', 'O1', 'P1', 'A2', 'B2']
 - 1.0 ul PB1 (C1)
          -> culture plate ['C2', 'D2', 'E2', 'F2', 'G2', 'H2', 'I2', 'J2', 'K2', 'L2', 'M2', 'N2', 'O2', 'P2', 'A3', 'B3', 'C3', 'D3', 'E3', 'F3', 'G3', 'H3', 'I3', 'J3', 'K12', 'L12', 'M12', 'N12', 'O12', 'P12', 'A13', 'B13', 'C13', 'D13', 'E13', 'F13', 'G13', 'H13', 'I13', 'J13', 'K13', 'L13', 'M13', 'N13', 'O13', 'P13', 'A14', 'B14']
 - 1.0 ul PB2 (D1)
          -> culture plate ['K3', 'L3', 'M3', 'N3', 'O3', 'P3', 'A4', 'B4', 'C4', 'D4', 'E4', 'F4', 'G4', 'H4', 'I4', 'J4', 'K4', 'L4', 'M4', 'N4', 'O4', 'P4', 'A5', 'B5', 'C14', 'D14', 'E14', 'F14', 'G14', 'H14', 'I14', 'J14', 'K14', 'L14', 'M14', 'N14', 'O14', 'P14', 'A15', 'B15', 'C15', 'D15', 'E15', 'F15', 'G15', 'H15', 'I15', 'J15']
 - 1.0 ul PB3 (E1)
          -> culture plate ['

# Shutdown

In [30]:
# allDone()
try:
    p20_left.drop_tip()
except:
    pass
try:
    p300_right.drop_tip()
except:
    pass
temp_mod.deactivate()
protocol.home()
#os.system("systemctl start opentrons-robot-server")

In [31]:
# complete tip usage check:
_=[display(show_rack_usage(labware)[1]) for labware in labware_list if 'Opentrons 96 Tip Rack' in str(labware)]

Opentrons 96 Tip Rack 20 µL on 8:
 -> 10 tips used:


,1,2,3,4,5,6,7,8,9,10,11,12
A,x,x,0,0,0,0,0,0,0,0,0,0
B,x,x,0,0,0,0,0,0,0,0,0,0
C,x,0,0,0,0,0,0,0,0,0,0,0
D,x,0,0,0,0,0,0,0,0,0,0,0
E,x,0,0,0,0,0,0,0,0,0,0,0
F,x,0,0,0,0,0,0,0,0,0,0,0
G,x,0,0,0,0,0,0,0,0,0,0,0
H,x,0,0,0,0,0,0,0,0,0,0,0


Opentrons 96 Tip Rack 20 µL on 11:
 -> 0 tips used:


,1,2,3,4,5,6,7,8,9,10,11,12
A,0,0,0,0,0,0,0,0,0,0,0,0
B,0,0,0,0,0,0,0,0,0,0,0,0
C,0,0,0,0,0,0,0,0,0,0,0,0
D,0,0,0,0,0,0,0,0,0,0,0,0
E,0,0,0,0,0,0,0,0,0,0,0,0
F,0,0,0,0,0,0,0,0,0,0,0,0
G,0,0,0,0,0,0,0,0,0,0,0,0
H,0,0,0,0,0,0,0,0,0,0,0,0


Opentrons 96 Tip Rack 300 µL on 10:
 -> 4 tips used:


,1,2,3,4,5,6,7,8,9,10,11,12
A,x,0,0,0,0,0,0,0,0,0,0,0
B,x,0,0,0,0,0,0,0,0,0,0,0
C,x,0,0,0,0,0,0,0,0,0,0,0
D,x,0,0,0,0,0,0,0,0,0,0,0
E,0,0,0,0,0,0,0,0,0,0,0,0
F,0,0,0,0,0,0,0,0,0,0,0,0
G,0,0,0,0,0,0,0,0,0,0,0,0
H,0,0,0,0,0,0,0,0,0,0,0,0
